In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import seaborn

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from models import build_heirarchical_logistic_regression_model
from utils import get_study_metrics_data


In [2]:
#studies_to_exclude = ['neurocube_quiroga_easy2', 'synth_mearec_neuronexus_noise10_K20_C32', 'neurocube_quiroga_easy1', 'neurocube_quiroga_difficult1', 'mea_c30', 'paired_monotrode_boyden32c', 'neurocube_sim2_11K20', 'paired_monotrode_mea64c', 'paired_english', 'synth_mearec_neuronexus_noise20_K40_C32', 'neurocube_quiroga_difficult2', 'paired_crcns', 'paired_monotrode_kampff', 'synth_mearec_neuronexus_noise20_K20_C32', 'synth_mearec_neuronexus_noise20_K10_C32', 'paired_kampff', 'paired_monotrode_crcns', 'synth_mearec_neuronexus_noise10_K10_C32', 'neurocube_sim2_2K10', 'paired_boyden32c', 'paired_mea64c', 'synth_mearec_neuronexus_noise10_K40_C32']
#ALL_AVAILABLE_STUDY_SETS=['HYBRID_JANELIA', 'LONG_DRIFT', 'LONG_STATIC', 'MANUAL_FRANKLAB', 'PAIRED_BOYDEN', 'PAIRED_CRCNS_HC1', 'PAIRED_ENGLISH', 'PAIRED_KAMPFF', 'PAIRED_MEA64C_YGER', 'PAIRED_MONOTRODE', 'SYNTH_BIONET', 'SYNTH_MAGLAND', 'SYNTH_MEAREC_NEURONEXUS', 'SYNTH_MEAREC_TETRODE', 'SYNTH_MONOTRODE', 'SYNTH_VISAPY']
#STUDY_NAMES = [study_name for study_set_name in ['HYBRID_JANELIA', 'LONG_STATIC', 'SYNTH_MAGLAND', 'SYNTH_MEAREC_TETRODE'] for study_name in SFStudySet.load(study_set_name).get_study_names() if study_name not in studies_to_exclude]


STUDY_NAMES = ['hybrid_static_tetrode', 'hybrid_static_siprobe',
               'LONG_STATIC_1200s_8c', 'LONG_STATIC_600s_8c', 'LONG_STATIC_300s_16c', 'LONG_STATIC_4800s_16c',
               'LONG_STATIC_300s_8c', 'LONG_STATIC_2400s_8c', 'LONG_STATIC_2400s_16c', 'LONG_STATIC_600s_16c',
               'LONG_STATIC_1200s_16c', 'LONG_STATIC_4800s_8c', 'synth_magland_noise20_K20_C8',
               'synth_magland_noise10_K10_C4', 'synth_magland_noise10_K10_C8', 'synth_magland_noise20_K10_C4',
               'synth_magland_noise20_K20_C4', 'synth_magland_noise20_K10_C8', 'synth_magland_noise10_K20_C8',
               'synth_magland_noise10_K20_C4', 'synth_mearec_tetrode_noise10_K20_C4', 'synth_mearec_tetrode_noise10_K10_C4',
               'synth_mearec_tetrode_noise20_K10_C4', 'synth_mearec_tetrode_noise20_K20_C4']

STATIC_SIPROBE_STUDY_NAMES = ['hybrid_static_siprobe', 'LONG_STATIC_1200s_8c', 'LONG_STATIC_600s_8c', 'LONG_STATIC_300s_16c',
                       'LONG_STATIC_4800s_16c', 'LONG_STATIC_300s_8c', 'LONG_STATIC_2400s_8c', 'LONG_STATIC_2400s_16c',
                       'LONG_STATIC_600s_16c', 'LONG_STATIC_1200s_16c', 'LONG_STATIC_4800s_8c',]

STATIC_TETRODE_STUDY_NAMES = ['hybrid_static_tetrode', 'synth_magland_noise20_K20_C8', 'synth_magland_noise10_K10_C4',
                       'synth_magland_noise10_K10_C8', 'synth_magland_noise20_K10_C4', 'synth_magland_noise20_K20_C4',
                       'synth_magland_noise20_K10_C8', 'synth_magland_noise10_K20_C8', 'synth_magland_noise10_K20_C4',
                       'synth_mearec_tetrode_noise10_K20_C4', 'synth_mearec_tetrode_noise10_K10_C4',
                       'synth_mearec_tetrode_noise20_K10_C4', 'synth_mearec_tetrode_noise20_K20_C4']

METRIC_NAMES = [ "firing_rate", "presence_ratio", "isi_violation",
                 "amplitude_cutoff", "snr", "max_drift", "cumulative_drift",
                 "silhouette_score", "isolation_distance", "l_ratio",
                 "nn_hit_rate", "nn_miss_rate", "d_prime"]

SORTER_NAMES = ['herdingspikes2', 'ironclust', 'jrclust',
               'kilosort', 'kilosort2', 'klusta', 'mountainsort4',
               'spykingcircus', 'tridesclous']

# SORTER_NAMES = ['ironclust', 'jrclust',
#                 'kilosort2', 'klusta', 'mountainsort4',
#                 'spykingcircus', 'tridesclous']

RANDOM_STATE = 0
P_VALUE = 0.01


In [3]:
static_tetrode_metric_data = get_study_metrics_data(
        study_names=STATIC_TETRODE_STUDY_NAMES,
        metric_names=METRIC_NAMES,
        random_state=RANDOM_STATE,
        sorter_names=SORTER_NAMES,
        # include_meta=True,
    ).dropna(axis=0)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(static_tetrode_metric_data.drop(columns=['fp']),
                                                    static_tetrode_metric_data['fp'], test_size=0.2, random_state=RANDOM_STATE)
X_train.describe()

,firing_rate,presence_ratio,isi_violation,amplitude_cutoff,snr,max_drift,cumulative_drift,silhouette_score,isolation_distance,l_ratio,nn_hit_rate,nn_miss_rate,d_prime
count,8931.000000,8931.000000,8931.000000,8931.000000,8931.000000,8931.000000,8931.000000,8931.000000,8.931000e+03,8931.000000,8931.000000,8931.000000,8931.000000
mean,3.653587,0.950386,0.887240,0.041703,8.826013,0.316307,1.264957,0.295213,5.294524e+08,0.321338,0.853053,0.009678,4.848100
std,5.294491,0.159769,7.183826,0.087653,4.807893,0.934494,4.161793,0.180365,3.639233e+10,1.147361,0.198849,0.017653,3.085005
min,0.020000,0.080000,0.000000,0.000862,0.277371,0.000000,0.000000,-0.267501,2.322603e+00,0.000000,0.019608,0.000000,0.573542
25%,1.694167,1.000000,0.000000,0.000863,5.409529,0.050000,0.160000,0.155312,2.883715e+01,0.000035,0.790399,0.001111,2.808032
50%,2.321667,1.000000,0.000000,0.001597,7.756750,0.090000,0.310000,0.290345,5.685787e+01,0.016245,0.953333,0.003464,4.041141
75%,2.727500,1.000000,0.246448,0.042307,11.219166,0.190000,0.610000,0.420786,1.203576e+02,0.239969,0.984444,0.011741,5.923032
max,57.316667,1.000000,382.319832,0.500000,43.737469,20.910000,110.080000,0.892608,3.210292e+12,27.095045,1.000000,0.593870,51.583097


In [5]:
standard_scalar = preprocessing.StandardScaler()
X_train = standard_scalar.fit_transform(X_train)
X_test = standard_scalar.transform(X_test)

In [6]:
lr_model = get_lr_model(X_train, y_train)
#%
with lr_model:
    trace = pm.sample(return_inferencedata=True)
    prior_checks = pm.sample_prior_predictive(trace, random_seed=RANDOM_STATE)


with lr_model:

    # change the value and shape of the data
    pm.set_data(
        {
            "lr_input": X_test,
            # use dummy values with the same shape:
            "lr_output": y_test,
        }
    )

    post_pred = pm.sample_posterior_predictive(trace.posterior)

NameError: name 'get_lr_model' is not defined

In [ ]:
from sklearn.metrics import f1_score

y_preds = [np.argmax(np.bincount(sample_pred)) for sample_pred in post_pred['out'].T]
f1_score(y_test, y_preds)

In [9]:
lr_model = build_heirarchical_logistic_regression_model(X_train, y_train)

with lr_model:
    trace = pm.sample(return_inferencedata=True)

    # change the value and shape of the data
    pm.set_data(
        {
            "X_observed": X_test,
            # use dummy values with the same shape:
            "y_observed": y_test,
        }
    )

    post_pred = pm.sample_posterior_predictive(trace.posterior)

/home/mclancy/.local/share/.pyenv/versions/3.8.3/envs/truespikes/lib/python3.8/site-packages/theano/tensor/basic.py:369: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.complex(data)  # works for all numeric scalars
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [bias, weights, bias_sigma, bias_mu, weights_sigma, weights_mu]
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 29 seconds.
There were 65 divergences after tuning. Increase `target_accept` or reparameterize.
There were 76 divergences after tuning. Increase `target_accept` or

In [11]:
from sklearn.metrics import f1_score

y_preds = [np.argmax(np.bincount(sample_pred)) for sample_pred in post_pred['y'].T]
f1_score(y_test, y_preds)

0.7983074753173484

In [ ]:
plt.figure(figsize=(9, 7))
seaborn.jointplot(trace["firing_rate"], trace["isolation_distance"], kind="hex", color="#4CB391")
plt.xlabel("beta_firing_rate")
plt.ylabel("beta_snr");

#